# Training a Machine Learning Classifier for HP-PPI Prediction Task

In [ ]:
import os
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from hyperopt import hp, tpe, STATUS_OK